In [1]:
import pandas as pd
import numpy as np 
from simulate import generateData

In [2]:
# number of observations
n=4000

# number of (Gaussian) covariates
p=20

# signal strength
signal_strength = 5

# Parameters for Distribution to draw Betas from. Betas ~ N(mu, stdev^2)
mu = 0
stdev = 1.0

X, Beta, y, means = generateData(dist = 'bernoulli', n = n, p = p, mu = mu, stdev = stdev, signal_strength = signal_strength) 

In [4]:
df = pd.DataFrame(X)
y = np.array(y)
y = np.where(y>0,1,-1)
df['y'] = pd.Series(y)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,y
0,0.030033,0.022415,0.015147,0.024897,-0.024281,0.006597,0.002650,0.014850,-0.014868,0.005297,...,-0.002732,-0.024734,0.004270,0.019432,0.018219,0.016328,-0.003915,0.017705,0.005061,1
1,-0.009963,-0.027749,0.019015,0.016912,0.008103,-0.008769,0.009538,0.009314,0.003776,0.014735,...,0.014670,0.002885,0.016106,-0.017062,0.018437,0.026280,0.002562,-0.006863,-0.009958,-1
2,-0.010501,-0.010404,0.006240,-0.014067,0.004493,0.015096,-0.024446,0.006141,0.001833,-0.016399,...,-0.012434,-0.032721,0.028799,0.017471,-0.017018,-0.003898,0.024457,0.015197,-0.017544,-1
3,0.007380,0.035462,0.007717,-0.003706,-0.009576,0.007025,-0.004750,-0.010335,0.013241,-0.008411,...,-0.032357,0.006662,0.003398,-0.002916,0.007785,0.010783,0.013633,0.001560,0.033495,1
4,-0.022545,-0.004905,0.005159,0.004043,-0.008269,0.006386,0.017574,-0.000058,-0.027601,0.014118,...,0.020152,0.007520,0.010139,0.015538,-0.002669,0.019355,0.001147,0.003607,0.015630,1


In [6]:
X = df.iloc[:,:-1]
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.030033,0.022415,0.015147,0.024897,-0.024281,0.006597,0.002650,0.014850,-0.014868,0.005297,0.017368,-0.002732,-0.024734,0.004270,0.019432,0.018219,0.016328,-0.003915,0.017705,0.005061
1,-0.009963,-0.027749,0.019015,0.016912,0.008103,-0.008769,0.009538,0.009314,0.003776,0.014735,0.007209,0.014670,0.002885,0.016106,-0.017062,0.018437,0.026280,0.002562,-0.006863,-0.009958
2,-0.010501,-0.010404,0.006240,-0.014067,0.004493,0.015096,-0.024446,0.006141,0.001833,-0.016399,0.001255,-0.012434,-0.032721,0.028799,0.017471,-0.017018,-0.003898,0.024457,0.015197,-0.017544
3,0.007380,0.035462,0.007717,-0.003706,-0.009576,0.007025,-0.004750,-0.010335,0.013241,-0.008411,0.003780,-0.032357,0.006662,0.003398,-0.002916,0.007785,0.010783,0.013633,0.001560,0.033495
4,-0.022545,-0.004905,0.005159,0.004043,-0.008269,0.006386,0.017574,-0.000058,-0.027601,0.014118,-0.011071,0.020152,0.007520,0.010139,0.015538,-0.002669,0.019355,0.001147,0.003607,0.015630


In [7]:
y = df.iloc[:,-1]
y

0       1
1      -1
2      -1
3       1
4       1
       ..
3995    1
3996    1
3997    1
3998    1
3999    1
Name: y, Length: 4000, dtype: int64

In [8]:
n,p = X.shape
print(n,p)

4000 20


In [9]:
from gurobipy import *

m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]
m.update()


float32_epsilon = (np.finfo(np.float32).eps)*10 
            ### We wanted strict inequality, but Gurobi can not handle strict inequality
            # So, Had to use machine epsilon as reference. However, machine epsilon turned out to be too small
            # So, we used ten times machine epsilon
            # Machine epsilon can be considered as the smallest number greater than zero.
            # https://kite.com/python/answers/how-to-find-machine-epsilon-using-numpy-in-python

# Constraints
for i in range(n):
    m.addConstr(y[i]*(X.iloc[i].dot(v))>= float32_epsilon)
m.update()


# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model

Using license file /home/nahid/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 4000 rows, 20 columns and 80000 nonzeros
Model fingerprint: 0x71004d66
Coefficient statistics:
  Matrix range     [4e-07, 7e-02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e-06]
Presolve time: 0.03s
Presolved: 20 rows, 4000 columns, 80000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   1.324058e-02      0s

Solved in 29 iterations and 0.03 seconds
Infeasible model
